In [13]:
import tensorflow as tf
from tensorflow import keras
from transformers import TFBertForSequenceClassification, BertTokenizer, TFBertModel
import pandas as pd
import numpy as np

from utils import load_sbic_dataset, load_validation_dataset

# Train the Model for SBIC dataset Using pretrained model bert

In [2]:
train_data = pd.read_csv("datasets/SBIC_train_processed.csv")
test_data = pd.read_csv("datasets/SBIC_test_processed.csv")
dev_data = pd.read_csv("datasets/SBIC_dev_processed.csv")

In [3]:
train_data.head()

,post,targetCategory_body,targetCategory_culture,targetCategory_disabled,targetCategory_gender,targetCategory_race,targetCategory_social,targetCategory_victim
0,bitch probably molly'd the 1st nigga that said...,False,False,False,True,False,False,False
1,bitch probably molly'd the 1st nigga that said...,False,False,False,True,False,False,False
2,bitch probably molly'd the 1st nigga that said...,False,False,False,False,True,False,False
3,bitch probably molly'd the 1st nigga that said...,False,False,False,False,True,False,False
4,RT @YungTweetGod: Bitches will give their puss...,False,False,False,True,False,False,False


In [4]:
# Load datasets (you'll need to implement this based on your data format)
train_texts, train_labels, test_texts, test_labels = load_sbic_dataset(
    train_path="datasets/SBIC_train_processed.csv",
    test_path="datasets/SBIC_test_processed.csv",
)
val_texts, val_labels = load_validation_dataset(
    dev_path="datasets/SBIC_dev_processed.csv"
)

print("train texts : ", train_texts[:2])
print("train labels : ", train_labels[:2])

train texts :  ["bitch probably molly'd the 1st nigga that said good mornin' back to her", "bitch probably molly'd the 1st nigga that said good mornin' back to her"]
train labels :  [[False False False  True False False False]
 [False False False  True False False False]]


In [14]:
# Load tokenizer and base BERT model
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertModel.from_pretrained(model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [15]:
# Define custom model for multi-label classification
class BiasDetectionModel(tf.keras.Model):
    def __init__(self, bert_model):
        super(BiasDetectionModel, self).__init__()
        self.bert = bert_model
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.classifier = tf.keras.layers.Dense(7, activation='sigmoid')

    def call(self, inputs):
        outputs = self.bert(inputs)[0]
        pooled_output = outputs[:, 0]
        pooled_output = self.dropout(pooled_output)
        return self.classifier(pooled_output)

# Instantiate the model
model = BiasDetectionModel(bert_model)

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=2e-5)
loss = keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=['binary_accuracy'])

# Tokenize and encode the datasets
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors="tf")
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128, return_tensors="tf")


In [7]:

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(1000).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(16)

In [8]:
# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)

Epoch 1/3
89/89 [==============================] - 1079s 11s/step - loss: 0.3034 - binary_accuracy: 0.8796 - val_loss: 0.2281 - val_binary_accuracy: 0.9185
Epoch 2/3
89/89 [==============================] - 607s 7s/step - loss: 0.1574 - binary_accuracy: 0.9397 - val_loss: 0.2148 - val_binary_accuracy: 0.9210
Epoch 3/3
89/89 [==============================] - 612s 7s/step - loss: 0.0992 - binary_accuracy: 0.9648 - val_loss: 0.2369 - val_binary_accuracy: 0.9159


In [9]:
# Save the model
model.save_weights('./bias_detection_model_tf/model_weights')
tokenizer.save_pretrained('./bias_detection_model_tf')

# Function for inference
def predict_bias(text):
    inputs = tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="tf")
    outputs = model(inputs)
    return outputs.numpy()[0]

In [10]:
# Example prediction
example_text = "Women belong in kitchen though."
predictions = predict_bias(example_text)
bias_categories = ['body', 'culture', 'disabled', 'gender', 'race', 'social', 'victim']
for category, prob in zip(bias_categories, predictions):
    print(f"{category}: {prob:.4f}")

body: 0.0020
culture: 0.0056
disabled: 0.0039
gender: 0.9822
race: 0.0040
social: 0.0040
victim: 0.0053


In [22]:
#Load the weights and test it with input test


model.load_weights('./bias_detection_model_tf/model_weights')

# Step 4: Define the inference function
def predict_bias(text):
    inputs = tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="tf")
    outputs = model(inputs['input_ids'])  # Feed the tokenized input into the model
    return outputs.numpy()[0]

# Step 5: Test with an example
example_text = "Women belong in kitchen though."
predictions = predict_bias(example_text)

# Step 6: Print predictions for each bias category
bias_categories = ['body', 'culture', 'disabled', 'gender', 'race', 'social', 'victim']
for category, prob in zip(bias_categories, predictions):
    print(f"{category}: {prob:.4f}")


body: 0.0020
culture: 0.0056
disabled: 0.0039
gender: 0.9822
race: 0.0040
social: 0.0040
victim: 0.0053


# Fine tuning the model 


In [118]:
df1=pd.read_csv("datasets/IndiBias_v1_sample.csv",usecols=lambda column: column != 'Unnamed: 0')
df2=pd.read_csv("datasets/crowsPairs/crows_pairs.csv",usecols=lambda column: column != 'Unnamed: 0')

In [119]:
df1.head()

,index,modified_eng_sent_more,modified_eng_sent_less,sent_more_hindi,sent_less_hindi,bias_type,stereo_antistereo
0,649,The brahmin family lived in a luxurious mansion.,The OBC family lived in a luxurious mansion.,ब्राह्मण परिवार एक आलीशान हवेली में रहता था।,ओबीसी परिवार एक आलीशान हवेली में रहता था।,Caste,stereo
1,667,The greedy landlord from vaishya community exp...,The greedy landlord from brahmin community exp...,वैश्य समुदाय के लालची मकान मालिक किरायेदारों क...,ब्राह्मण समुदाय के लालची जमींदार किरायेदारों क...,Caste,stereo
2,588,Some people assume that being SC/ST means you'...,Some people assume that being Brahmin means yo...,कुछ लोग मानते हैं कि अनुसूचित जाति होने का मतल...,कुछ लोग मानते हैं कि ब्राह्मण होने का मतलब है ...,Caste,antistereo
3,650,The speaker argued that brahmin students does ...,The speaker argued that OBC students does not ...,वक्ता ने तर्क दिया कि ब्राह्मण छात्रों को गुणव...,वक्ता ने तर्क दिया कि ओबीसी छात्रों को गुणवत्त...,Caste,antistereo
4,665,You can't trust those vaishya traders; their g...,You can't trust those kshatriya traders; their...,आप उन वैश्य व्यापारियों पर भरोसा नहीं कर सकते;...,आप उन क्षत्रिय व्यापारियों पर भरोसा नहीं कर सक...,Caste,stereo


In [120]:
df2.head()

,sent_more,sent_less,stereo_antistereo,bias_type,annotations,anon_writer,anon_annotators
0,He couldn't figure out the issue with the rope...,He couldn't figure out the issue with the rope...,stereo,race-color,"[['race-color'], ['race-color'], ['race-color'...",a0,"['a77', 'a78', 'a79', 'a80', 'a81']"
1,The poor are really ignorant about how to hand...,The rich are really ignorant about how to hand...,stereo,socioeconomic,"[['socioeconomic'], ['socioeconomic'], ['socio...",a1,"['a20', 'a54', 'a82', 'a83', 'a84']"
2,His mind wondered if a doctor was behind this ...,His mind wondered if a doctor was behind this ...,antistereo,gender,"[['gender'], ['gender'], ['gender'], [], ['rac...",a2,"['a85', 'a13', 'a7', 'a3', 'a79']"
3,the girl was sprawled out and it was obvious a...,the girl was sprawled out and it was obvious a...,stereo,race-color,"[['race-color'], ['race-color'], ['race-color'...",a3,"['a86', 'a87', 'a88', 'a37', 'a89']"
4,Asians cant drive because they have slanted eyes,White people cant drive because they have slan...,stereo,race-color,"[['race-color'], ['race-color'], ['race-color'...",a4,"['a90', 'a91', 'a5', 'a92', 'a93']"


In [121]:
# Extract relevant columns from Dataset 1 (IndiBias)
train_data_1 = df1[['modified_eng_sent_more', 'modified_eng_sent_less', 'bias_type']].copy()

# Rename columns for consistency (optional)
train_data_1.rename(columns={'modified_eng_sent_more': 'sent_more', 
                            'modified_eng_sent_less': 'sent_less'}, inplace=True)
train_data_1.head()

,sent_more,sent_less,bias_type
0,The brahmin family lived in a luxurious mansion.,The OBC family lived in a luxurious mansion.,Caste
1,The greedy landlord from vaishya community exp...,The greedy landlord from brahmin community exp...,Caste
2,Some people assume that being SC/ST means you'...,Some people assume that being Brahmin means yo...,Caste
3,The speaker argued that brahmin students does ...,The speaker argued that OBC students does not ...,Caste
4,You can't trust those vaishya traders; their g...,You can't trust those kshatriya traders; their...,Caste


In [122]:
train_data_2 = df2[['sent_more', 'sent_less', 'bias_type']].copy()
train_data_2.head()

,sent_more,sent_less,bias_type
0,He couldn't figure out the issue with the rope...,He couldn't figure out the issue with the rope...,race-color
1,The poor are really ignorant about how to hand...,The rich are really ignorant about how to hand...,socioeconomic
2,His mind wondered if a doctor was behind this ...,His mind wondered if a doctor was behind this ...,gender
3,the girl was sprawled out and it was obvious a...,the girl was sprawled out and it was obvious a...,race-color
4,Asians cant drive because they have slanted eyes,White people cant drive because they have slan...,race-color


In [123]:
train_df = pd.concat([train_data_1,train_data_2], ignore_index=True)
train_df.head()

,sent_more,sent_less,bias_type
0,The brahmin family lived in a luxurious mansion.,The OBC family lived in a luxurious mansion.,Caste
1,The greedy landlord from vaishya community exp...,The greedy landlord from brahmin community exp...,Caste
2,Some people assume that being SC/ST means you'...,Some people assume that being Brahmin means yo...,Caste
3,The speaker argued that brahmin students does ...,The speaker argued that OBC students does not ...,Caste
4,You can't trust those vaishya traders; their g...,You can't trust those kshatriya traders; their...,Caste


In [124]:
train_df['bias_type'].unique()

array(['Caste', 'Religion', 'age', 'disability', 'gender',
       'physical-appearance', 'socioeconomic', 'race-color',
       'nationality', 'sexual-orientation', 'religion'], dtype=object)

In [103]:
len(train_df)

2069

In [104]:
train_df.isnull().sum()

sent_more    0
sent_less    0
bias_type    0
dtype: int64

In [105]:
#Preprocess Train data

# Drop duplicates if any
train_df.drop_duplicates(inplace=True)

# Reset index after cleaning
train_df.reset_index(drop=True, inplace=True)

In [106]:
len(train_df)

2067

In [125]:
# Load the saved weights from the SBIC-trained model
model.load_weights('./bias_detection_model_tf/model_weights')


In [131]:
# Load the tokenizer for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.classifier = tf.keras.layers.Dense(11, activation='softmax')  # Adjust to 11 output neurons


# Compile the modified model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [183]:
# Tokenize the sentences from the combined dataset
def tokenize_sentences(sentences, tokenizer, max_length=128):
    return tokenizer(sentences, truncation=True, padding='max_length', max_length=max_length, return_tensors="tf")

# Tokenize the sentences
tokenized_inputs = tokenize_sentences(train_df['sent_more'].tolist(), tokenizer)

# Convert bias_type to labels (assuming it's categorical)
train_df['bias_type'] = train_df['bias_type'].astype('category').cat.codes
labels = tf.convert_to_tensor(train_df['bias_type'], dtype=tf.int32)

In [184]:
# Determine the number of samples
total_samples = len(train_df)

# Slice the dataset
train_val_encodings = {key: value[:-200] for key, value in tokenized_inputs.items()}  # All but last 200 for training/validation
test_encodings = {key: value[-200:] for key, value in tokenized_inputs.items()}  # Last 200 for testing

# Slice the labels
train_val_labels = labels[:-200]  # First part for training/validation
test_labels = labels[-200:]  # Last 200 for testing


In [185]:
# Define how many samples to use for validation (10% of the remaining samples)
val_size = int(0.1 * (total_samples - 200))

# Slice train/validation sets
train_encodings = {key: value[:-val_size] for key, value in train_val_encodings.items()}
val_encodings = {key: value[-val_size:] for key, value in train_val_encodings.items()}

train_labels = train_val_labels[:-val_size]
val_labels = train_val_labels[-val_size:]


In [169]:
batch_size = 16

# Create TensorFlow datasets for training, validation, and testing
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(1000).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(batch_size)


In [176]:
train_dataset

<BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [138]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Fine-tune the model
history = model.fit(
    train_dataset,               # Training dataset
    validation_data=val_dataset,  # Validation dataset
    epochs=3                      # Adjust epochs as needed
)


Epoch 1/3
130/130 [==============================] - 742s 6s/step - loss: 0.8986 - accuracy: 0.7216 - val_loss: 0.2417 - val_accuracy: 0.9409
Epoch 2/3
130/130 [==============================] - 938s 7s/step - loss: 0.3144 - accuracy: 0.9004 - val_loss: 0.0959 - val_accuracy: 0.9677
Epoch 3/3
130/130 [==============================] - 1008s 8s/step - loss: 0.1523 - accuracy: 0.9531 - val_loss: 0.0450 - val_accuracy: 0.9946


In [139]:
# Evaluate the model on the test dataset (200 samples)
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


13/13 [==============================] - 27s 2s/step - loss: 0.1050 - accuracy: 0.9550
Test Loss: 0.10504811257123947, Test Accuracy: 0.9549999833106995


In [192]:
# Step 4: Define the inference function
def predict_bias(text):
    inputs = tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="tf")
    outputs = model(inputs['input_ids'])  # Feed the tokenized input into the model
    return outputs.numpy()[0]

# Step 5: Test with an example
example_text = "Women belong in kitchen though."
predictions = predict_bias(example_text)

# Step 6: Print predictions for each bias category
bias_categories = ['Caste', 'Religion', 'age', 'disability', 'gender',
       'physical-appearance', 'socioeconomic', 'race-color',
       'nationality', 'sexual-orientation', 'religion']
for category, prob in zip(bias_categories, predictions):
    print(f"{category}: {prob:.4f}")

Caste: 0.0000
Religion: 0.0000
age: 0.0000
disability: 0.0002
gender: 0.9975
physical-appearance: 0.0000
socioeconomic: 0.0001
race-color: 0.0000
nationality: 0.0000
sexual-orientation: 0.0020
religion: 0.0001


In [189]:
model.save_weights('./fine_tune_model_tf/model_weights')
tokenizer.save_pretrained('./fine_tune_model_tf')

('./fine_tune_model_tf/tokenizer_config.json',
 './fine_tune_model_tf/special_tokens_map.json',
 './fine_tune_model_tf/vocab.txt',
 './fine_tune_model_tf/added_tokens.json')